[H2O AutoML](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html) is an automated machine learning meta-algorithm that is part of the [H2O software library](http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/intro.html#what-is-h2o). (It shold not be confused with [H2O DriverlessAI](https://www.h2o.ai/products/h2o-driverless-ai/), which is a commercial product and built from an entirely different code base.) H2O’s AutoML can be used for automating the machine learning workflow, which includes automatic training and tuning of many models within a user-specified time-limit. Stacked Ensembles – one based on all previously trained models, another one on the best model of each family – will be automatically trained on collections of individual models to produce highly predictive ensemble models which, in most cases, will be the top performing models in the AutoML Leaderboard.

In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/ieee-fraud-detection"))
print(os.listdir("../input/standalone-train-and-test-preprocessing"))

# Any results you write to the current directory are saved as output.

['ieee-fraud-detection', 'standalone-train-and-test-preprocessing']
['test_identity.csv', 'sample_submission.csv', 'train_identity.csv', 'train_transaction.csv', 'test_transaction.csv']
['object_columns.npy', '__notebook__.ipynb', '__output__.json', 'train.csv', 'custom.css', 'test.csv', '__results__.html']


In [11]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

3.26.0.5
Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,4 mins 16 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.5
H2O cluster version age:,1 month and 2 days
H2O cluster name:,H2O_from_python_unknownUser_z54h4d
H2O cluster total nodes:,1
H2O cluster free memory:,14.21 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [9]:
!cd ..
!ls -la

total 16
drwxr-xr-x 3 root root 4096 Oct 19 14:20 .
drwxr-xr-x 6 root root 4096 Oct 19 14:19 ..
drwxr-xr-x 2 root root 4096 Oct 19 14:20 .ipynb_checkpoints
-rw-r--r-- 1 root root  199 Oct 19 14:19 __notebook_source__.ipynb


In [17]:
%%time
train = h2o.import_file("../input/standalone-train-and-test-preprocessing/train.csv")
#test = h2o.import_file("../input/fork-of-ieee-lgbm-with-groupkfold-cv/test.csv")
#train = train[:250000,:]

Parse progress: |█████████████████████████████████████████████████████████| 100%
CPU times: user 452 ms, sys: 60 ms, total: 512 ms
Wall time: 24.7 s


In [19]:
test = h2o.import_file("../input/standalone-train-and-test-preprocessing/test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [18]:
train.head()

isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146
0,86400,68.5,W,13926,nan,150,discover,142,credit,315,87,19,nan,,,1,1,0,0,0,1,0,0,1,0,2,0,1,1,14,nan,13,nan,nan,nan,nan,nan,nan,13,13,nan,nan,nan,0,T,T,T,M2,F,T,,,,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1,1,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,117,0,0,0,0,0,117,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,86401,29,W,2755,404,150,mastercard,102,credit,325,87,nan,nan,gmail.com,,1,1,0,0,0,1,0,0,0,0,1,0,1,1,0,nan,nan,0,nan,nan,nan,nan,nan,0,nan,nan,nan,nan,0,,,,M0,T,T,,,,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,1,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,86469,59,W,4663,490,150,visa,166,debit,330,87,287,nan,outlook.com,,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0,nan,nan,0,nan,nan,nan,nan,nan,0,315,nan,nan,nan,315,T,T,T,M0,F,F,F,F,F,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,1,1,1,1,0,0,0,0,0,1,1,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,86499,50,W,18132,567,150,mastercard,117,debit,476,87,nan,nan,yahoo.com,,2,5,0,0,0,4,0,0,1,0,1,0,25,1,112,112,0,94,0,nan,nan,nan,nan,84,nan,nan,nan,nan,111,,,,M0,T,F,,,,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1,1,1,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,1,1,1,1,0,0,0,0,0,1,1,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,1,48,28,0,10,4,1,38,24,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,50,1758,925,0,354,135,50,1404,790,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,86506,50,H,4497,514,150,mastercard,102,credit,420,87,nan,nan,gmail.com,,1,1,0,0,0,1,0,1,0,1,1,0,1,1,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,,,,,,,,,,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,18,140,0
0,86510,49,W,5937,555,150,visa,226,debit,272,87,36,nan,gmail.com,,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0,nan,nan,0,nan,nan,nan,nan,nan,0,0,nan,nan,nan,0,T,T,T,M1,F,T,,,,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,1,1,1,1,0,0,0,0,0,1,1,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,n

In [20]:
test.head()

TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147
1.84032e+07,31.95,W,10409,111,150,visa,226,debit,170,87,1,nan,gmail.com,,6,6,0,0,3,4,0,0,6,0,5,1,115,6,419,419,27,398,27,nan,nan,nan,nan,418,203,nan,nan,nan,409,T,T,F,,,F,T,T,T,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,1,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,47.95,0,0,47.95,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1.84033e+07,49,W,4272,111,150,visa,226,debit,299,87,4,nan,aol.com,,3,2,0,0,0,1,0,0,2,0,1,1,12,2,149,149,7,634,7,nan,nan,nan,nan,231,634,nan,nan,nan,634,T,F,F,M0,,F,,,,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,4,1,0,4,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,280,77,0,280,77,0,0,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1.84033e+07,171,W,4476,574,150,visa,226,debit,472,87,2635,nan,hotmail.com,,2,2,0,0,0,5,0,0,4,0,2,0,22,2,137,137,10,97,10,nan,nan,nan,nan,136,136,nan,nan,nan,97,T,T,F,M0,F,F,F,F,F,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,1,0,0,2,2,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,1,2,1,2,0,0,0,1,2,0,0,1,1,1,0,0,0,0,0,0,0,5,0,0,3,0,0,0,0,0,2,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,968,0,0,705,0,0,0,0,0,263,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1.84033e+07,284.95,W,10989,360,150,visa,166,debit,205,87,17,nan,gmail.com,,5,2,0,0,1,1,0,0,2,0,2,0,7,4,42,42,41,242,41,nan,nan,nan,nan,242,242,nan,nan,nan,242,T,T,T,,,T,,,,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,0,0,0,2,2,2,2,0,0,1,0,0,2,2,2,2,1,1,0,0,0,1,1,1,1,0,0,0,0,1,1,0,0,1,1,1,0,1,1,0,0,0,0,1,1,1,1,0,0,0,1,1,0,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1.84033e+07,67.95,W,18018,452,150,mastercard,117,debit,264,87,6,nan,gmail.com,,6,6,0,0,2,5,0,0,5,0,6,0,14,6,22,22,0,22,0,nan,nan,nan,nan,22,22,nan,nan,nan,22,T,T,T,,,F,F,T,T,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,0,0,0,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,0,0,0,1,1,1,1,0,0,0,0,1,1,0,0,1,1,1,0,1,1,0,0,0,0,1,1,1,1,0,0,0,1,1,0,0,1,1,1,0,1,1,0,0,0,1,3,1,1,3,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,67.95,183.85,67.95,67.95,183.85,67.95,0,0,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1.84033e+07,57.95,W,12839,321,150,visa,226,debit,512,87,nan,nan,gmail.com,,5,5,0,0,2,3,0,0,2,0,4,0,10,4,36,36,35,0,nan,nan,nan,nan,nan,0,0,nan,nan,nan,0,T,T,T,,,T,F,F,T,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,0,0,0,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0,1,1,1,1,0,0,0,1,1,0,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1.84034e+07,87,W,16560,476,150,visa,126,debit,110,87,nan,nan,gmail.com,,1,1,0,0,0,1,0,0,1,0,1,0,2,1,nan,nan,0,0,nan,nan,nan,nan,nan,0,0,nan,nan,nan,0,T,T

In [21]:
object_columns = np.load('../input/standalone-train-and-test-preprocessing/object_columns.npy')

In [22]:
object_columns

array(['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain',
       'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12',
       'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'], dtype='<U13')

In [ ]:
#for f in object_columns:
#    train[f] = train[f].asfactor()
#    test[f] = test[f].asfactor()

In [23]:
x = train.columns[:-1]
y = 'isFraud'
# For binary classification, response should be a factor
train[y] = train[y].asfactor()

In [ ]:
aml = H2OAutoML(max_models=8, seed=47, max_runtime_secs=18000)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |██

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [ ]:
# The leader model is stored here
aml.leader

In [ ]:
h2o.remove(train)
test = h2o.import_file("../input/fork-of-ieee-lgbm-with-groupkfold-cv/test.csv")

In [ ]:
preds = aml.predict(test)
preds['p1'].as_data_frame().values.flatten().shape


In [ ]:
sample_submission = pd.read_csv('../input/ieee-fraud-detection/sample_submission.csv')
sample_submission.shape

In [ ]:
sample_submission['isFraud'] = preds['p1'].as_data_frame().values
sample_submission.to_csv('h2o_automl_submission_5.csv', index=False)